In [82]:
from scipy.spatial import cKDTree

def get_kd_tree_of_crack_surface(filename):
    points = []
    with open(filename, 'r') as r:
        r.readline()
        for line in r:
            tokens = line.split(',')
            x = float(tokens[3])
            y = float(tokens[4])
            z = float(tokens[5].strip())
            points.append((x,y,z))
    
    kd_tree = cKDTree(points)
    return kd_tree

In [99]:
import numpy as np

def find_normal_of_plane_for_point(tree, x,y,z):
    distances, indices = tree.query((x,y,z),3)
    pt1 = tree.data[indices[0]]
    pt2 = tree.data[indices[1]]
    pt3 = tree.data[indices[2]]
    from1to2 = ((pt2[0]-pt1[0], pt2[1]-pt1[1], pt2[2]-pt1[2]))
    from1to3 = ((pt3[0]-pt1[0], pt3[1]-pt1[1], pt3[2]-pt1[2]))
    normal = np.cross(from1to3, from1to2)
    if normal[0] == 0 and normal[1] == 0 and normal[2] == 0:
        distances, indices = tree.query((x,y,z),3) # Tried this with 4 and it worked but I think 3 will be better with the crack front point...
        pt1 = ((x,y,z))
        pt2 = tree.data[indices[1]]
        pt3 = tree.data[indices[2]]
        from1to2 = ((pt2[0]-pt1[0], pt2[1]-pt1[1], pt2[2]-pt1[2]))
        from1to3 = ((pt3[0]-pt1[0], pt3[1]-pt1[1], pt3[2]-pt1[2]))
        normal_ = np.cross(from1to3, from1to2)
        return normal_
    return normal

In [92]:
import numpy as np
from math import acos

def find_raw_angle_between_vectors(x1, y1, z1, x2, y2, z2):
    dot = np.dot((x1,y1,z1), (x2, y2, z2))
    magnitude1 = (x1**2 + y1**2 + z1**2)**.5
    magnitude2 = (x2**2 + y2**2 + z2**2)**.5
    return acos(dot / (magnitude1 * magnitude2))

In [85]:
from math import pi

def convert_raw_to_scaled(angle):
    if angle > pi / 2:
        return pi - angle
    else:
        return angle

In [86]:
tree = get_kd_tree_of_crack_surface('../data/0-D1_grid_points_with_attributes_transformed.csv')
print('done')

In [100]:
with open('../data/4-phis-misorientation.csv', 'r') as r:
    with open('../data/5-crack-surface-normal.csv', 'w') as w:
        # Write the file headers
        headers = ('crack_id,theta,x,y,z,dadN3Dline,grain_id,nearest_grain_boundary_x,nearest_grain_boundary_y,'
                   'nearest_grain_boundary_z,nearest_grain_boundary_id,distance_to_grain_boundary,'
                   'vector_to_grain_boundary_x,vector_to_grain_boundary_y,vector_to_grain_boundary_z,'
                   'magnitude_of_vector_to_grain_boundary,unit_vector_to_grain_boundary_x,unit_vector_to_grain_boundary_y,'
                   'unit_vector_to_grain_boundary_z,phi1,Phi,phi2,'
                   'nearest_grain_phi1,nearest_grain_Phi,nearest_grain_phi2,'
                   'misorientation,normal_x,normal_y,normal_z,raw_angle,scaled_angle\n')
        w.write(headers)
        
        # Skip the headers of the file
        r.readline()
        
        for line in r:
            # Read in the tokens
            tokens = line.split(',')
            
            crack_id = tokens[0]
            theta = tokens[1]
            
            x = float(tokens[2])
            y = float(tokens[3])
            z = float(tokens[4])
            dadN3Dline = tokens[5]
            grain_id = int(tokens[6])
            
            nearest_grain_boundary_x = tokens[7]
            nearest_grain_boundary_y = tokens[8]
            nearest_grain_boundary_z = tokens[9]
            nearest_grain_boundary_id = int(tokens[10])
            
            distance_to_grain_boundary = tokens[11]
            vector_to_grain_boundary_x = float(tokens[12])
            vector_to_grain_boundary_y = float(tokens[13])
            vector_to_grain_boundary_z = float(tokens[14])
            magnitude_of_vector_to_grain_boundary = tokens[15]
            unit_vector_to_grain_boundary_x = tokens[16]
            unit_vector_to_grain_boundary_y = tokens[17]
            unit_vector_to_grain_boundary_z = tokens[18]
            
            phi1 = tokens[19]
            Phi = tokens[20]
            phi2 = tokens[21]
            nearest_grain_phi1 = tokens[22]
            nearest_grain_Phi = tokens[23]
            nearest_grain_phi2 = tokens[24]
            misorientation = tokens[25].strip()

            # Find the normal vector to the plane for the point on the crack surface
            normal_x, normal_y, normal_z = find_normal_of_plane_for_point(tree, x, y, z)

            # Get the raw angle between the crack surface normal vector and the vector to the nearest grain boundary
            raw_angle = find_raw_angle_between_vectors(vector_to_grain_boundary_x, vector_to_grain_boundary_y, vector_to_grain_boundary_z, 
                                                       normal_x, normal_y, normal_z)
            
            scaled_angle = convert_raw_to_scaled(raw_angle)

            # Write it to the file
            to_write = '{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n'\
                                .format(crack_id,theta,x,y,z,
                                dadN3Dline,grain_id,nearest_grain_boundary_x,nearest_grain_boundary_y,
                                nearest_grain_boundary_z,nearest_grain_boundary_id,distance_to_grain_boundary,
                                vector_to_grain_boundary_x,vector_to_grain_boundary_y,vector_to_grain_boundary_z,
                                magnitude_of_vector_to_grain_boundary,unit_vector_to_grain_boundary_x,
                                unit_vector_to_grain_boundary_y, unit_vector_to_grain_boundary_z, phi1, Phi, phi2, 
                                nearest_grain_phi1, nearest_grain_Phi, nearest_grain_phi2, misorientation, 
                                normal_x, normal_y, normal_z, raw_angle, scaled_angle)
            w.write(to_write)
print('done')

done
